<a href="https://colab.research.google.com/github/HegdeSiddesh/cs6910_Assignment3/blob/main/Transliteration_using_fairseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference for this: https://towardsdatascience.com/implementing-transformer-for-language-modeling-ba5dd60389a2

In [ ]:
# Install the necessary libraries
!pip3 install sacremoses pandas mock sacrebleu tensorboardX pyarrow xformers triton #indic-nlp-library

# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq

!pip install --editable ./
%cd .. 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 15.8 MB/s 
     |████████████████████████████████| 92 kB 12.9 MB/s 
     |████████████████████████████████| 125 kB 34.6 MB/s 
     |████████████████████████████████| 219 kB 70.3 MB/s 
     |████████████████████████████████| 18.2 MB 55.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=0bfa755c192e2c53452f35ca3f76dc7ac419a67025431915d4a826fdf50d9566
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
  Created wheel for xformers: filename=xformers-0.0.11-cp37-cp37m-linux_x86_64.whl size=6111712 sha256=9d1dcbc1ac7ba180701d6b382077e1392359d9bae710122d37aa5204d6c6c632
  Stored in directory: /root/.cache/pip/wheels/cd/68/bc/dfcdbce20dbf08da5f0ae62a75f270f20b2be0c8bb0f8c1587
Successfully built sacremoses xformers
Cloning into 'fairseq'...
remo

In [ ]:
!pip install triton==2.0.0.dev20220430

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.3 MB 137 kB/s 
  Attempting uninstall: triton
    Found existing installation: triton 1.1.1
    Uninstalling triton-1.1.1:
      Successfully uninstalled triton-1.1.1


In [ ]:
import torch
import numpy as np
import pandas as pd

In [ ]:
# adding path to env variable
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

In [ ]:
#Download and unzip the Dakshina dataset 
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf '/content/dakshina_dataset_v1.0.tar'

--2022-07-20 06:23:34--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.128, 172.217.194.128, 74.125.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  49.8MB/s    in 35s     

2022-07-20 06:24:10 (54.3 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



### Data preprocessing and cleaning

In [ ]:
# creating the corpus of files with appropriate format
%mkdir corpus

In [ ]:
# all the training, testing, validation files will be there in corpus. in the format: train_$lang_abr.en, train_$lang_abr.$lang_abr
lang_abr_list = ['hi']
lang_list = ['Hindi']

all_lang_train_lines_nat = []
all_lang_train_lines_rom = []


for lang, lang_abr in zip(lang_list, lang_abr_list): #(Hindi, hi)
    
    print("lang : ",lang)

    ######PREPROCESSING FOR THE TEST DATA
    f_in_test = open('/content/dakshina_dataset_v1.0/'+lang_abr+'/lexicons/'+lang_abr+'.translit.sampled.test.tsv', 'r')

    # Variable to store the list of lines from the above file
    lines_in_test = f_in_test.read().split('\n')

    f_in_test.close()

    lines_in_test = [line for line in lines_in_test if line ]

    print("Unpreprocessed:",lines_in_test[0])

    #Create these 2 new files to store the test data
    f_test_nat_out = open('corpus/test_'+lang_abr+'.'+lang_abr,'w')
    f_test_rom_out = open('corpus/test_'+lang_abr+'.en','w')

    #Word and english transliteration are separated by tab
    #Here we collect the list of english and native words in appropriate lists
    lines_test_nat = [ ' '.join(line.split('\t')[0]) for line in lines_in_test ]
    lines_test_rom = [ ' '.join(line.split('\t')[1]) for line in lines_in_test ]

    print("Preprocessed:",lines_test_nat[0])
    print("Preprocessed:",lines_test_rom[0])

    #Store above words in the appropriate files for each language
    f_test_nat_out.write('\n'.join(lines_test_nat))
    f_test_rom_out.write('\n'.join(lines_test_rom))

    #Close the files
    f_test_nat_out.close()
    f_test_rom_out.close()

    ######        REPEAT ABOVE PROCESS FOR THE VALIDATION DATA          
    f_in_valid = open('/content/dakshina_dataset_v1.0/'+lang_abr+'/lexicons/'+lang_abr+'.translit.sampled.dev.tsv', 'r')
    lines_in_valid = f_in_valid.read().split('\n')
    f_in_valid.close()

    lines_in_valid = [line for line in lines_in_valid if line ]

    f_valid_nat_out = open('corpus/valid_'+lang_abr+'.'+lang_abr,'w')
    f_valid_rom_out = open('corpus/valid_'+lang_abr+'.en','w')

    lines_valid_nat = [ ' '.join(line.split('\t')[0]) for line in lines_in_valid ]
    lines_valid_rom = [ ' '.join(line.split('\t')[1]) for line in lines_in_valid ]

    f_valid_nat_out.write('\n'.join(lines_valid_nat))
    f_valid_rom_out.write('\n'.join(lines_valid_rom))

    f_valid_nat_out.close()
    f_valid_rom_out.close()



    ######        REPEAT ABOVE PROCESS FOR THE TRAIN DATA
    f_in = open('/content/dakshina_dataset_v1.0/'+lang_abr+'/lexicons/'+lang_abr+'.translit.sampled.train.tsv', 'r')
    lines_in = f_in.read().split('\n')
    f_in.close()

    lines_in = [line for line in lines_in if line ]


    f_train_nat_out = open('corpus/train_'+lang_abr+'.'+lang_abr,'w')
    f_train_rom_out = open('corpus/train_'+lang_abr+'.en','w')

    lines_train_nat = [ ' '.join(line.split('\t')[0]) for line in lines_in ]
    lines_train_rom = [ ' '.join(line.split('\t')[1]) for line in lines_in ]

    f_train_nat_out.write('\n'.join(lines_train_nat))
    f_train_rom_out.write('\n'.join(lines_train_rom))

    f_train_nat_out.close()
    f_train_rom_out.close()

    all_lang_train_lines_nat += lines_train_nat
    all_lang_train_lines_rom += lines_train_rom



print('writing files to the train corpus')
# writing files to the corpus
f_out_train_nat = open('corpus/train_combine.cmb','w')
f_out_train_rom = open('corpus/train_combine.en','w')
f_out_train_nat.write('\n'.join(all_lang_train_lines_nat))
f_out_train_rom.write('\n'.join(all_lang_train_lines_rom))
f_out_train_nat.close()
f_out_train_rom.close()


lang :  Hindi
Unpreprocessed: अंक	ank	5
Preprocessed: अ ं क
Preprocessed: a n k
writing files to the train corpus


### Vocab creation

Reference: https://fairseq.readthedocs.io/en/latest/command_line_tools.html

In [ ]:
  # --trainpref	train file prefix (also used to build dictionaries)
  # -s, --source-lang	source language
  # -t, --target-lang	target language
  # --workers	number of parallel workers ,Default: 1
  # --destdir	destination dir Default: “data-bin”


!fairseq-preprocess \
  --trainpref corpus/train_combine  \
  --source-lang en --target-lang cmb \
  --workers 256 \
  --destdir corpus-bin

2022-07-20 06:24:21 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='corpus-bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, simul_type=None, source_lang='en',

### Binarizing the corpus

In [ ]:
#Binarizing the data (creating a binary vector representation for data)

# binarizing the files using the created dictionaries
# --srcdict	reuse given source dictionary
# --tgtdict	reuse given target dictionary
%%shell
for lang_abr in hi
do
   fairseq-preprocess \
   --trainpref corpus/train_$lang_abr \
   --validpref corpus/valid_$lang_abr \
   --testpref corpus/test_$lang_abr \
   --srcdict corpus-bin/dict.en.txt \
   --tgtdict corpus-bin/dict.cmb.txt \
   --source-lang en --target-lang $lang_abr \
   --workers 32 \
   --destdir corpus-bin 
done

2022-07-20 06:24:41 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='corpus-bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, simul_type=None, source_lang='en',

In [ ]:
#this syntax is to run a shell script
%%shell  

# adding all the languages to the lang_list.txt
echo "hi" >> lang_list.txt
echo "en" >> lang_list.txt

### Training

In [ ]:
# training script
#fairseq-train : Train a new model on one or across multiple GPUs

## REFERENCE: https://github.com/facebookresearch/fairseq/tree/main/examples/translation#multilingual-translation
# Choose architecture out of the possible choices
# Choose task. Default is translation(see doc)
# Temperature sampling: https://towardsdatascience.com/how-to-sample-from-language-models-682bceb97277


!fairseq-train corpus-bin \
  --save-dir transformer \
  --arch transformer --layernorm-embedding \
  --task translation_multi_simple_epoch \
  --sampling-method "temperature" \
  --sampling-temperature 1.5 \
  --encoder-langtok "tgt" \
  --lang-dict lang_list.txt \
  --lang-pairs en-hi  \
  --decoder-normalize-before --encoder-normalize-before \
  --activation-fn gelu --adam-betas "(0.9, 0.98)"  \
  --batch-size 1024 \
  --decoder-attention-heads 4 --decoder-embed-dim 256 --decoder-ffn-embed-dim 1024 --decoder-layers 6 \
  --dropout 0.5 \
  --encoder-attention-heads 4 --encoder-embed-dim 256 --encoder-ffn-embed-dim 1024 --encoder-layers 6 \
  --lr 0.001 --lr-scheduler inverse_sqrt \
  --max-epoch 120 \
  --optimizer adam  \
  --num-workers 32 \
  --warmup-init-lr 0 --warmup-updates 4000

Streaming output truncated to the last 5000 lines.
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
OSError: [Errno 9] Bad file descriptor
Exception ignored in: <function _MultiProcessingDataLoade

### Result generation

In [ ]:
%mkdir output

mkdir: cannot create directory ‘output’: File exists


In [ ]:
# generation script
#Explanation for S,D,P,H in the en-hi output file


%%shell
for lang_abr in hi
do
source_lang=en
target_lang=$lang_abr
fairseq-generate corpus-bin \
  --path transformer/checkpoint_best.pt \
  --task translation_multi_simple_epoch \
  --gen-subset test \
  --beam 4 \
  --nbest 4 \
  --source-lang $source_lang \
  --target-lang $target_lang \
  --batch-size 4096 \
  --encoder-langtok "tgt" \
  --lang-dict lang_list.txt \
  --num-workers 64 \
  --lang-pairs en-hi  > output/${source_lang}_${target_lang}.txt
done


2022-07-20 11:42:55 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name'

In [ ]:
res_dict = {}

lang = 'Hindi'
lang_abr = 'hi'

f = open('output/en_'+lang_abr+'.txt','r')
lines = f.read().split('\n')

list_s = [line for line in lines if 'S-' in line]
list_t = [line for line in lines if 'T-' in line]
list_h = [line for line in lines if 'H-' in line]
list_d = [line for line in lines if 'D-' in line]

for s in list_s:
    s_id = int(s.split('\t')[0].split('-')[1])
    
    res_dict[s_id] = { 'S' : s.split('\t')[1] }
    
    for t in list_t:
        t_id = int(t.split('\t')[0].split('-')[1])
        if s_id == t_id:
            res_dict[s_id]['T'] = t.split('\t')[1] 

    res_dict[s_id]['H'] = []
    res_dict[s_id]['D'] = []
    
    for h in list_h:
        h_id = int(h.split('\t')[0].split('-')[1])

        if s_id == h_id:
            res_dict[s_id]['H'].append( ( h.split('\t')[2], pow(2,float(h.split('\t')[1])) ) )
    
    for d in list_d:
        d_id = int(d.split('\t')[0].split('-')[1])
    
        if s_id == d_id:
            res_dict[s_id]['D'].append( ( d.split('\t')[2], pow(2,float(d.split('\t')[1]))  ) )


for r in res_dict.keys():
    res_dict[r]['H'].sort(key = lambda x : float(x[1]) ,reverse =True)
    res_dict[r]['D'].sort(key = lambda x : float(x[1]) ,reverse =True)


In [ ]:
res_dict_train = {}
res_dict_test = {}


for i in res_dict.keys():

    source_text = res_dict[i]['S']
    res_dict_train[source_text] = list()
    

    res_dict_test[source_text] = list()
    res_dict_test[source_text].append(res_dict[i]['T'])
    for j in range(len(res_dict[i]['H'])):
        
        target_text = res_dict[i]['H'][j][0]
        res_dict_train[source_text].append(target_text)

In [ ]:
import json

with open('train_data_hi.json', 'w') as fp:
    json.dump(res_dict_train, fp)

with open('test_data_hi.json', 'w') as fp:
    json.dump(res_dict_test, fp)

In [ ]:
import json


with open('train_data_hi.json', 'r') as fp:
    res_dict_train = json.load(fp)


with open('test_data_hi.json', 'r') as fp:
    res_dict_test = json.load(fp)

## Metrics for computing accuracy

In [ ]:
## Reference: https://towardsdatascience.com/evaluating-ocr-output-quality-with-character-error-rate-cer-and-word-error-rate-wer-853175297510

def levenshtein(u, v):
    prev = None
    curr = [0] + list(range(1, len(v) + 1))
    # Operations: (SUB, DEL, INS)
    prev_ops = None
    curr_ops = [(0, 0, i) for i in range(len(v) + 1)]
    for x in range(1, len(u) + 1):
        prev, curr = curr, [x] + ([None] * len(v))
        prev_ops, curr_ops = curr_ops, [(0, x, 0)] + ([None] * len(v))
        for y in range(1, len(v) + 1):
            delcost = prev[y] + 1
            addcost = curr[y - 1] + 1
            subcost = prev[y - 1] + int(u[x - 1] != v[y - 1])
            curr[y] = min(subcost, delcost, addcost)
            if curr[y] == subcost:
                (n_s, n_d, n_i) = prev_ops[y - 1]
                curr_ops[y] = (n_s + int(u[x - 1] != v[y - 1]), n_d, n_i)
            elif curr[y] == delcost:
                (n_s, n_d, n_i) = prev_ops[y]
                curr_ops[y] = (n_s, n_d + 1, n_i)
            else:
                (n_s, n_d, n_i) = curr_ops[y - 1]
                curr_ops[y] = (n_s, n_d, n_i + 1)
    return curr[len(v)], curr_ops[len(v)]

    

def charerr(ref,hyp):
    _, (s, i, d) = levenshtein(ref, hyp)
    cer_s = s
    cer_i = i
    cer_d = d
    cer_n = len(ref)

    if cer_n > 0:
        return (cer_s + cer_i + cer_d) / cer_n
    else:
        return 0

In [ ]:
# !sudo apt-get install fastwer

# import fastwer
#!pip install git+https://github.com/huggingface/evaluate

#!pip install jiwer jiwer

# from evaluate import load
# cer = load("cer")
# cer_score = cer.compute(predictions="abc", references="abd")

In [ ]:
def evaluate(input_data, test_data):
    '''
    Evaluates and returns char and word level accuracy
    '''
    from evaluate import load
    cer = load("cer")
    acc = {}
    characc ={}
    characc2 = {}
    charAccCount = 0
    charAccCount2 = 0
    correctWordCount = 0
    correctCount = 0
    totalWords = 0
    
    for src_word in list(test_data.keys()):
        if src_word in input_data:
            totalWords = totalWords+1
            candidates = input_data[src_word]
            references = test_data[src_word]

            #compute word accuracy
            acc[src_word] = max([int(candidates[0] == ref) for ref in references]) # either 1 or 0
            correctWordCount = correctWordCount + acc[src_word]

            # compute char accuracy
            characc[src_word] = 1 - charerr(references[0], candidates[0])
            charAccCount = charAccCount + characc[src_word]

            # Computation using library
            # characc2[src_word] = 1 - cer.compute(predictions=[candidates[0]], references=[references[0]])
            # charAccCount2 = charAccCount2 + characc2[src_word]
            # print(characc2[src_word])

        else:
            print('No transliterations')
            characc[src_word] = 0.0

    return acc, characc, (correctWordCount/totalWords), (charAccCount/totalWords)#, characc2, (charAccCount2/totalWords)

In [ ]:
acc, characc, wordLevelAcc, charLevelAcc = evaluate(res_dict_train, res_dict_test)

In [ ]:
print(characc)

{'__hi__ k h a n i k a r m a': 0.6923076923076923, '__hi__ s a b m i s s i o n': 0.2727272727272727, '__hi__ s u b m i s s i o n': 0.8181818181818181, '__hi__ s a m a a d h a a n': 1.0, '__hi__ s a m a l a k h a a': 0.8181818181818181, '__hi__ s a a t h s a a t h': 1.0, '__hi__ a s h a k t a t a a': 1.0, '__hi__ a s h r v a t a t h': 0.7692307692307692, '__hi__ a a l o c h a k o n': 1.0, '__hi__ c o n v e n t i o n': 1.0, '__hi__ s h u b h a n k a r': 1.0, '__hi__ k h a n i k a r m i': 0.7692307692307692, '__hi__ g i r i j a g h a r': 0.8461538461538461, '__hi__ c h a u d h a v a n': 0.6923076923076923, '__hi__ c h a u d h a w a n': 0.6923076923076923, '__hi__ c h a u s i n g h a': 0.9230769230769231, '__hi__ j a n a j a g r a n': 0.8461538461538461, '__hi__ j a n j a a g r a n': 0.8461538461538461, '__hi__ j a y a l a l i t a': 0.8461538461538461, '__hi__ d h a m k a a t a a': 1.0, '__hi__ b a i t h a n t e y': 0.4444444444444444, '__hi__ a a p a d a a a e n': 1.0, '__hi__ u t h a a p

In [ ]:
print(acc)

{'__hi__ k h a n i k a r m a': 0, '__hi__ s a b m i s s i o n': 0, '__hi__ s u b m i s s i o n': 0, '__hi__ s a m a a d h a a n': 1, '__hi__ s a m a l a k h a a': 0, '__hi__ s a a t h s a a t h': 1, '__hi__ a s h a k t a t a a': 1, '__hi__ a s h r v a t a t h': 0, '__hi__ a a l o c h a k o n': 1, '__hi__ c o n v e n t i o n': 1, '__hi__ s h u b h a n k a r': 1, '__hi__ k h a n i k a r m i': 0, '__hi__ g i r i j a g h a r': 0, '__hi__ c h a u d h a v a n': 0, '__hi__ c h a u d h a w a n': 0, '__hi__ c h a u s i n g h a': 0, '__hi__ j a n a j a g r a n': 0, '__hi__ j a n j a a g r a n': 0, '__hi__ j a y a l a l i t a': 0, '__hi__ d h a m k a a t a a': 1, '__hi__ b a i t h a n t e y': 0, '__hi__ a a p a d a a a e n': 1, '__hi__ u t h a a p a t a k': 0, '__hi__ k a u s h a l a t a': 1, '__hi__ k h i n c h a k a r': 0, '__hi__ c h h a v i y a a n': 0, '__hi__ c h h e t t h a r i': 0, '__hi__ d h o o n d h a t a': 0, '__hi__ a n a v a s h y a k': 1, '__hi__ d h a a t u y e i n': 1, '__hi__ p

In [ ]:
print("Char Level Acc:", charLevelAcc)

Char Level Acc: 0.8667970568404578


In [ ]:
#print("Word Level Acc:", wordLevelAcc)

In [ ]:
#print("Char Level Acc:", charAccCount2)